### COLAB ONLY

If you're on COLAB, run the next cells.


**THEN** after you create the `spark` session you can run the following:

```
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
```

and in another cell (wait 10 sec to `ngrok` to start):

```
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
```

This will show a URL where to find the Spark UI.

**BEWARE** not stable. Connection limits problem. In case wait a minute before browsing again.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark
from google.colab import files

In [ ]:
files.upload()

# Spark DATAFRAMES

This notebook is about spark dataframes


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [2]:
datafile = '2015-summary.csv'

In [3]:
s = SparkSession.builder.master('local[*]').appName('Flights').getOrCreate()

In [4]:
sc = s.sparkContext

In [5]:
sc

<SparkContext master=local[*] appName=Flights>

In [9]:
flightData2015 = (s.
                 read.
                 option('inferSchema', 'true').
                 option('header', 'true').
                 csv(datafile)
                 )

In [10]:
flightData2015

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [16]:
%%bash
!head 2015-summary.csv

Couldn't find program: 'bash'


In [11]:
flightData2015.rdd

MapPartitionsRDD[29] at javaToPython at NativeMethodAccessorImpl.java:0

In [12]:
flightData2015.show(4)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
+-----------------+-------------------+-----+
only showing top 4 rows



In [13]:
# create new dataframe by applying a transformation or action on top of the dataframe

a = flightData2015.sort('count')

In [14]:
a.show(4)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
+--------------------+-------------------+-----+
only showing top 4 rows



In [15]:
flightData2015.sort('count').explain()

== Physical Plan ==
*(1) Sort [count#43 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#43 ASC NULLS FIRST, 200), true, [id=#69]
   +- FileScan csv [DEST_COUNTRY_NAME#41,ORIGIN_COUNTRY_NAME#42,count#43] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/quent/Documents/Programming/UCA_MSc-DS1/big_data_python/lecture_..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [16]:
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [17]:
flightData2015.take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]

In [18]:
dataFrameway = flightData2015.groupby('DEST_COUNTRY_NAME').count()

In [19]:
dataFrameway.take(5)

[Row(DEST_COUNTRY_NAME='Anguilla', count=1),
 Row(DEST_COUNTRY_NAME='Russia', count=1),
 Row(DEST_COUNTRY_NAME='Paraguay', count=1),
 Row(DEST_COUNTRY_NAME='Senegal', count=1),
 Row(DEST_COUNTRY_NAME='Sweden', count=1)]

In [20]:
dataFrameway.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#41], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#41, 200), true, [id=#125]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#41], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#41] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/quent/Documents/Programming/UCA_MSc-DS1/big_data_python/lecture_..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [21]:
flightData2015.createOrReplaceTempView('flight_data_table_2015')

In [24]:
sqlway = s.sql("""
SELECT DEST_COUNTRY_NAME, COUNT(1) FROM flight_data_table_2015 GROUP BY DEST_COUNTRY_NAME
""")

In [25]:
sqlway.take(5)

[Row(DEST_COUNTRY_NAME='Anguilla', count(1)=1),
 Row(DEST_COUNTRY_NAME='Russia', count(1)=1),
 Row(DEST_COUNTRY_NAME='Paraguay', count(1)=1),
 Row(DEST_COUNTRY_NAME='Senegal', count(1)=1),
 Row(DEST_COUNTRY_NAME='Sweden', count(1)=1)]

In [26]:
sqlway.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#41], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#41, 200), true, [id=#171]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#41], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#41] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/quent/Documents/Programming/UCA_MSc-DS1/big_data_python/lecture_..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [27]:
s.sql("SELECT MAX(COUNT) FROM flight_data_table_2015").take(1)

[Row(max(COUNT)=370002)]

In [28]:
from pyspark.sql.functions import max

In [29]:
flightData2015.select(max('count')).take(1)

[Row(max(count)=370002)]

In [30]:
maxSql = s.sql("""
SELECT DEST_COUNTRY_NAME, sum(COUNT) as dest_total
FROM flight_data_table_2015 
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

In [32]:
maxSql.show()

+-----------------+----------+
|DEST_COUNTRY_NAME|dest_total|
+-----------------+----------+
|    United States|    411352|
|           Canada|      8399|
|           Mexico|      7140|
|   United Kingdom|      2025|
|            Japan|      1548|
+-----------------+----------+



In [33]:
from pyspark.sql.functions import desc

In [34]:
maxDFWay = (flightData2015
            .groupby("DEST_COUNTRY_NAME")
            .sum('count')
            .withColumnRenamed('sum(count)', 'dest_total')
            .sort(desc('dest_total'))
            .limit(5))

In [35]:
maxDFWay.show()

+-----------------+----------+
|DEST_COUNTRY_NAME|dest_total|
+-----------------+----------+
|    United States|    411352|
|           Canada|      8399|
|           Mexico|      7140|
|   United Kingdom|      2025|
|            Japan|      1548|
+-----------------+----------+

